# Data Intuition (Cont'd)

Putting it all together:
- pandas
- data wrangling
- data intuition
- descriptive statistics
- wisdom of the crowds

We'll use your responses to the Fermi estimation Google Form to do this.

## Setup

In [1]:
# import working with data libraries
import pandas as pd
import numpy as np

## The Data

In [2]:
# read data into Python
df = pd.read_csv('data/fermi_fa20.csv')

In [3]:
# take a look at the data
df

,Timestamp,How fast does human hair grow (cm/yr)?,"If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?","How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?"
0,10/10/20 2:37,20,3500000,10
1,10/10/20 13:16,25,1 million,50
2,10/10/20 13:45,10^3 cm/yr,10^7 km^2,10^4
3,10/10/20 13:52,6cm/yr,1000km^2,200 days
4,10/10/20 14:18,81,16875000000,28000
...,...,...,...,...
186,10/16/20 18:00,12,650,"42 (I'm assuming ""here"" means ucsd)"
187,10/16/20 20:05,10,2500000,1000
188,10/16/20 22:32,7,100000,240
189,10/16/20 23:04,20,500000,40


## Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### Thought Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

## Brainstorming

What considerations do we have to make about these data?

- handling non-numeric answers (i.e. "lots")
- handle units in answer
- unreasonably large/small number (outliers)? (not thoughtful, or joking)
- scientific notation

In [4]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

,timestamp,hair_growth,crammed,SAN_NYC
0,10/10/20 2:37,20,3500000,10
1,10/10/20 13:16,25,1 million,50
2,10/10/20 13:45,10^3 cm/yr,10^7 km^2,10^4
3,10/10/20 13:52,6cm/yr,1000km^2,200 days
4,10/10/20 14:18,81,16875000000,28000


In [5]:
# check type of each Series (column)
df.dtypes

timestamp      object
hair_growth    object
crammed        object
SAN_NYC        object
dtype: object

In [6]:
# this cell will produce an error
df['hair_growth'].median()

TypeError: 

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

In [7]:
# take a look at unique values
df["hair_growth"].unique()

array(['20', '25', '10^3 cm/yr', '6cm/yr', '81', '15', '10^2', '3',
       '10 cm/yr', '12', '30', '5', '100', '10', '24', '10^1', '19',
       '16 cm/yr', '17', '48 cm/yr', '10cm/yr', '60', '65', '7',
       '6 cm/yr', '30 cm/yr', '60cm/yr', '50', '18', '36 cm/yr', '8',
       '36', '10cm', '40 cm', '15cm/yr', '15 cm/yr', '1ft per year',
       '7 cm', '100cm/yr', '10 cm/year', '70', '15 cm', '17.78', '4', '2',
       '10cm/year', '1000000', '200', '7 cm/yr', '265', '0.1', '40',
       '20 cm/yr', '60/1', '6', '80', '14', '5cm/yr', '16', '10^1 cm'],
      dtype=object)

In [8]:
# standardize height column
def standardize_hair(string):
    
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()

    # take care of unique cases
    string = string.replace("60/1", "60")
    string = string.replace("1ft per year", "30.48")
    string = string.replace("10^1.2", "15.8")
    string = string.replace("10^(1.2)", "15.8")
    string = string.replace("0.5 in/month", "6")
    
    # remove commas
    string = string.replace(",", "")
    
    # take care of inculded unit cases
    string = string.replace("cm/year", "")
    string = string.replace("cm/yr", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")

    # handle scientific notation
    string = string.replace("^3", "00")
    string = string.replace("^2", "0")
    string = string.replace("^1", "")
    string = string.replace("10^(-2)", "0.1")
    string = string.replace("10^-2", "0.1")

    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [9]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

array([2.000e+01, 2.500e+01, 1.000e+03, 6.000e+00, 8.100e+01, 1.500e+01,
       1.000e+02, 3.000e+00, 1.000e+01, 1.200e+01, 3.000e+01, 5.000e+00,
       2.400e+01, 1.900e+01, 1.600e+01, 1.700e+01, 4.800e+01, 6.000e+01,
       6.500e+01, 7.000e+00, 5.000e+01, 1.800e+01, 3.600e+01, 8.000e+00,
       4.000e+01, 3.048e+01, 7.000e+01, 1.778e+01, 4.000e+00, 2.000e+00,
       1.000e+06, 2.000e+02, 2.650e+02, 1.000e-01, 8.000e+01, 1.400e+01])

In [10]:
df['hair_growth'].describe()

count        191.000000
mean        5267.690262
std        72355.168002
min            0.100000
25%           10.000000
50%           16.000000
75%           30.000000
max      1000000.000000
Name: hair_growth, dtype: float64

In [11]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

16.0

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

In [12]:
df['crammed'].unique()

array(['3500000', '1 million', '10^7 km^2', '1000km^2', '16875000000',
       '10', '10^2', '250,000', '800000', '5,000,000', '7 * 10^7', '8000',
       '750,000km^2', '10^4', '100', '10000', '10^5', '6,000,000,000,000',
       '10^8', '1,000,000', '1000', '100,000', '4000',
       '7,000,000,000,000', '7000000000', '1000000', '7,000,000,000',
       '10^6', '100,000 km squared', '15', '599km^2', '3.5x10^9 km',
       '7 x 10^7', '800', '1600000', '10000000', '10^5 km ^2', '2.4x10^7',
       '130000 km squared', '4 quadrillion', '1500', '2133600000', '600',
       '100,000,000', '700', '200', '100000', '10,000,000', '1000000km²',
       '7000 km^2', '7 billion', '7mill km^2', '1.13 * 10^12',
       '7 billion km^2', '2000', '2km square ', '3.2 * 10^10 km^2', '650',
       '750,000', '1oo km^2', '10,000,000 km^2', '50', '2000000',
       '2.4 km^2', 'The size of California ', '5 x 10^5', '10^10',
       '1.39E-07', '10km^2', '10^3', '2500 km^2', '4200', '7.14x10^7',
       '5,334,000 km

In [13]:
# standardize crammed column
def standardize_crammed(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of included unit cases
    string = string.replace("(km^2)", "")
    string = string.replace("km^2", "")
    string = string.replace("km ^2", "")
    string = string.replace("km2", "")
    string = string.replace("km²", "")
    string = string.replace("km", "")
    string = string.replace("squared", "")
    string = string.replace("square", "")
    string = string.replace("sq", "")
    
    # take care of various scientific notation cases
    string = string.replace("1.13 * 10^12", "1130000000000")
    string = string.replace("2.4x10^7", "24000000")
    string = string.replace("3.5x10^9 ", "3500000000")
    string = string.replace("2.4 *10", "24")
    string = string.replace("3.2 * 10^10 ", "32000000000")
    string = string.replace("1oo", "100")
    string = string.replace("7.14x10^7", "71400000")
    string = string.replace("7x(10^3)", "7000")
    string = string.replace("3.6 * 10^13", "36000000000000")
    string = string.replace("3.5*10^4", "35000")
    string = string.replace("1.0 x 10^10", "10000000000")
   
    # take care of general scientific notation
    string = string.replace("x10^2", "00")
    string = string.replace("*(10^3)", "000")
    string = string.replace("10^2", "100")
    string = string.replace("10^3", "1000")
    string = string.replace("10^4", "10000")
    string = string.replace("*10^5", "00000")
    string = string.replace(" x 10^5", "00000")
    string = string.replace("10^5", "100000")
    string = string.replace("*10^6", "000000")
    string = string.replace(" × 10^6", "000000")
    string = string.replace("10^6", "1000000")
    string = string.replace(" * 10^7", "0000000")
    string = string.replace(" x 10^7", "0000000")
    string = string.replace("10^7", "10000000")
    string = string.replace("*10^8", "100000000")
    string = string.replace("10^8", "100000000")
    string = string.replace(" x 10^9", "000000000")
    string = string.replace("x 10^9", "000000000")
    string = string.replace("10^9", "1000000000")
    string = string.replace("10^10", "1000000000")
    string = string.replace("10^11", "00000000000")
    string = string.replace("10^15", "1000000000000000")
    string = string.replace(" x 10^9", "000000000")
    string = string.replace("10x1000000", "10000000")

    
    # take care of unique word cases
    string = string.replace("10^-3 per person 1000000000 people on earth answer: [1000000]", "1000000")
    string = string.replace(" million", "000000")
    string = string.replace("mill", "000000")
    string = string.replace(" billion", "000000000")
    string = string.replace(" quadrillion", "000000000000000")
    string = string.replace("the size of california", "423970")

    # convert to numeric
    output = float(string)
    
    return output

In [14]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

array([3.5000e+06, 1.0000e+06, 1.0000e+07, 1.0000e+03, 1.6875e+10,
       1.0000e+01, 1.0000e+02, 2.5000e+05, 8.0000e+05, 5.0000e+06,
       7.0000e+07, 8.0000e+03, 7.5000e+05, 1.0000e+04, 1.0000e+05,
       6.0000e+12, 1.0000e+08, 4.0000e+03, 7.0000e+12, 7.0000e+09,
       1.5000e+01, 5.9900e+02, 3.5000e+09, 8.0000e+02, 1.6000e+06,
       2.4000e+07, 1.3000e+05, 4.0000e+15, 1.5000e+03, 2.1336e+09,
       6.0000e+02, 7.0000e+02, 2.0000e+02, 7.0000e+03, 7.0000e+06,
       1.1300e+12, 2.0000e+03, 2.0000e+00, 3.2000e+10, 6.5000e+02,
       5.0000e+01, 2.0000e+06, 2.4000e+00, 4.2397e+05, 5.0000e+05,
       1.0000e+09, 1.3900e-07, 2.5000e+03, 4.2000e+03, 7.1400e+07,
       5.3340e+06, 9.0000e+03, 2.0000e+16, 1.7500e+03, 2.0000e+04,
       0.0000e+00, 3.6000e+13, 4.0000e+06, 3.5000e+04, 5.0000e+07,
       9.0000e+06, 1.5000e+05, 2.0000e+07, 5.2500e+03, 1.0000e+10,
       2.5000e+04, 8.0000e+07, 5.8000e+05, 6.5000e+01, 2.5000e+01,
       1.4700e+04, 1.1500e+06, 1.1000e+11, 4.0000e+07, 2.0000e

In [15]:
df.dtypes

timestamp       object
hair_growth    float64
crammed        float64
SAN_NYC         object
dtype: object

In [16]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

100000.0

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [17]:
df['SAN_NYC'].unique()

array(['10', '50', '10^4', '200 days ', '28000', '200', '10^3', '100',
       '29 days', '21', '2 years', '300', '180', 'forever', '35', '20',
       '45', '12', '30', '150', '365', '78', '81 days', '42', '8 days',
       '100 days (at 30mi/day)', '31', '35 days', '25 days', '36',
       '1 week', '500', '120', '14', '100,000', '100 days', '200 Days',
       '90', '300 days', '1000', '6 months', '30 days ', '38 days', '40',
       '1000 days ', '30 days', '140', '36000 minutes', '950', '400',
       '2 months ', '7', '10^2', '2 days', '11 days', '17 days', '182',
       '1 1/2 months', '9', '2 months', '48', '14 days', '70 days', '60',
       '250', '36,500', '41', '15', '833', '46', '40 days', '52', '70',
       '69', '280', '3 years', '43', '1200', '1,500', '28', '1.8 * 10',
       '3 months', '25', '50 days', '10^5', '6 days ', '1300',
       '42 (I\'m assuming "here" means ucsd)', '240'], dtype=object)

In [18]:
# standardize distance column
def standardize_distance(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of special/unclear cases
    string = string.replace("forever", "NaN")
    string = string.replace('42 (i\'m assuming "here" means ucsd)', "42")
    string = string.replace("100 days (at 30mi/day)", "100")
    string = string.replace("1.8 * 10", "18")

    # remove units
    string = string.replace("days", "")

    # take care of non-days answers
    string = string.replace("3 years", "1095")
    string = string.replace("2 years", "730")
    string = string.replace("1 1/2 months", "45")
    string = string.replace("3 months", "92")
    string = string.replace("2 months", "61")
    string = string.replace("6 months", "180")
    string = string.replace("1 week", "7")
    string = string.replace("36000 minutes", "25")

    
    # take care of 'punctuation'
    string = string.replace(",", "")
    
    # take care of scientific notation 
    string = string.replace("10^2", "100")
    string = string.replace("10^3", "1000")
    string = string.replace("10^4", "10000")
    string = string.replace("10^5", "100000")

    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [19]:
# apply function across values in crammed columns
df["SAN_NYC"] = df["SAN_NYC"].apply(standardize_distance)
df["SAN_NYC"].unique()

array([1.000e+01, 5.000e+01, 1.000e+04, 2.000e+02, 2.800e+04, 1.000e+03,
       1.000e+02, 2.900e+01, 2.100e+01, 7.300e+02, 3.000e+02, 1.800e+02,
             nan, 3.500e+01, 2.000e+01, 4.500e+01, 1.200e+01, 3.000e+01,
       1.500e+02, 3.650e+02, 7.800e+01, 8.100e+01, 4.200e+01, 8.000e+00,
       3.100e+01, 2.500e+01, 3.600e+01, 7.000e+00, 5.000e+02, 1.200e+02,
       1.400e+01, 1.000e+05, 9.000e+01, 3.800e+01, 4.000e+01, 1.400e+02,
       9.500e+02, 4.000e+02, 6.100e+01, 2.000e+00, 1.100e+01, 1.700e+01,
       1.820e+02, 9.000e+00, 4.800e+01, 7.000e+01, 6.000e+01, 2.500e+02,
       3.650e+04, 4.100e+01, 1.500e+01, 8.330e+02, 4.600e+01, 5.200e+01,
       6.900e+01, 2.800e+02, 1.095e+03, 4.300e+01, 1.200e+03, 1.500e+03,
       2.800e+01, 1.800e+01, 9.200e+01, 6.000e+00, 1.300e+03, 2.400e+02])

In [20]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

69.5

## Take Home Messages

1. Look at your data! (People behave unexpectedly.)
2. Get your data into a tidy data format.
3. Fix (Standardize) your data - or, better yet, collect better data!
4. Use your intuition/knowledge/information out there to consider if values make sense.